In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, ShuffleSplit, RepeatedKFold
import numpy as np
from itertools import combinations
import statsmodels.api as sm
import statsmodels.tools as st
from statsmodels.stats.outliers_influence import variance_inflation_factor
import time
import matplotlib.pyplot as plt
import pprint
import statistics
import time
%matplotlib inline

In [40]:
response = 'rheomix final deg time min'
# response = 'rheomix stability time min'

In [41]:
file1 = 'Total_model_subset_{}_AIC_BIC_AICc.csv'.format(response)
df_AIC = pd.read_csv(file1)

In [56]:
CN_lim = 1500
VIF_lim = 100

In [57]:
df_AIC_filter = df_AIC[(df_AIC['Cond_No'] < CN_lim)].copy()
df_AIC_filter2 = df_AIC_filter[(df_AIC_filter['AICc'] == df_AIC_filter['AICc'].min())].copy()

In [58]:
def import_data(experimental_file, response_file, response):
    'imports data'
    'experimental_file = experimental design csv filename i.e experimental.csv'
    'response_file = results csv filename i.e Response.csv'
    'response = reponse name: i.e rheomix final deg time min or rheomix stability time min'
    
    experimental_df = pd.read_csv(experimental_file)
    response_df = pd.read_csv(response_file)
    
    X = experimental_df[experimental_df.columns.values.tolist()[1:]].values
    y = response_df[response].values
    max1 = max(y)
    min1 = min(y)

    y_norm = [2*((i-min1)/(max1-min1)) - 1 for i in y]
    
    
    X_linear = X
    linear_terms = experimental_df.columns.values.tolist()[1:]
    
    return y_norm, X_linear, linear_terms, experimental_df, response_df

def X_gen(model, X_linear):
    
    for i, j in enumerate(model):
        
        if i == 0 and len(j) == 2: 
            X_new = X_linear[:, model[0][1]]
        
        else:
            
            if len(j) == 2:
                add = X_linear[:, j[1]]

            if len(j) == 3:
                add = X_linear[:, j[1]]*X_linear[:, j[2]]

            X_new2 = np.column_stack((X_new, add))
            X_new = X_new2

    
    return X_new

In [59]:
dict1 = {'Xpvc': 0, 'Xfiller': 1, 'Xfr': 2, 'Xstabiliser': 3, 'Xdinp': 4, 'Xldh':5, "Xsph": 6}

experimental_file = 'experimental.csv'
response_file = 'Response.csv'

y, X_linear, linear_terms, experimental_df, response_df = import_data(experimental_file, response_file, response)

model = df_AIC_filter2['Model'].values[0]

    
cnt = 0                  
a = model.split('--')[1:-1]
i = []
for b in a:
    if len(b.split('*')) == 1:
        i.append([b, dict1[b]])
    else:
        i.append([b, dict1[b.split('*')[0]], dict1[b.split('*')[1]]])

X = X_gen(i, X_linear)

model_fit = sm.OLS(y, X)
results = model_fit.fit()
variables = model_fit.exog
COND = results.condition_number
AIC = st.eval_measures.aicc(results.llf, results.nobs, results.df_model)
if len(i) != 1:

    vif = [variance_inflation_factor(variables, m) for m in range(variables.shape[1])]
    vif_max = max(vif)

else:
    vif_max = 0

print('Model:  ',model, '\n\nVIF: ', vif_max, '\n\nCOND: ', COND, '\n\nAICc: ', AIC)
print(results.summary())

Model:   --Xpvc--Xfiller--Xstabiliser--Xdinp--Xldh--Xstabiliser*Xldh--Xdinp*Xldh-- 

VIF:  49.76600214772193 

COND:  1493.511191412442 

AICc:  16.53370090020981
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.886
Model:                            OLS   Adj. R-squared (uncentered):              0.866
Method:                 Least Squares   F-statistic:                              43.51
Date:                Sun, 24 Oct 2021   Prob (F-statistic):                    1.76e-16
Time:                        11:45:59   Log-Likelihood:                         0.20683
No. Observations:                  46   AIC:                                      13.59
Df Residuals:                      39   BIC:                                      26.39
Df Model:                           7                                                  
Covariance Type:            nonrobust        

In [50]:
df_AIC_filter2

,Unnamed: 0,Unnamed: 0.1,Model,No_terms,AIC,AICc,BIC,Cond_No
44,44,44,--Xdinp--Xldh--Xdinp*Xldh--,3,19.928825,20.500253,25.414749,80.747958
